In [1]:
#!pip install -U swifter

In [2]:
# !pip install -U pandas
# !pip install -U seaborn

In [3]:
#!sed -i ':a;N;$!ba;s/\,\n/\,/g' results/Cameron*

In [4]:
import re
import pandas as pd
import analysis_relabel_funcs
from analysis import read_dfs
import torch
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from torch.nn import Softmax
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
import pickle
import swifter
import time
%matplotlib inline

In [5]:
MODEL_RELABEL = {
    'BERT-Jigsaw': ["not toxic", "toxic"], 
    'BERT-SBIC-offensive': ["Not", "Maybe", "Offensive"],
    'BERT-SBIC-targetcategory': ['none', 'body', 'culture', 'disabled', 'gender', 'race', 'social', 'victim'], # not used
    'BERT-eec-emotion': ['none', 'anger','fear', 'joy', 'sadness'], # not used
    'BERT-jigsaw-identityhate': ['Not', "Yes"],
    'BERT-jigsaw-severetoxic': ["Not", "Yes"],
    'BERT-mdgender-convai-binary': ["female", "male"],
    'BERT-mdgender-convai-ternary': ["female", "male", "neutral"],
    'BERT-mdgender-wizard': ["neutral", "female", "male"],
    'BERT-rtgender-opgender-annotations': ["man", "woman"] 
}

In [6]:
def _softmax_and_relabel(predictions, categorical_labels):
    # takes in a torch.Tensor of predictions and a list of categorical labels, returns a tuple of (softmax tensor, categorical label)
    m = Softmax(dim=0)
    sm = m(predictions)
    return sm, categorical_labels[sm.argmax().item()]

def score(labels, predictions):
    softmax_preds, category = _softmax_and_relabel(predictions, labels)
    # scores are always the difference between the first and last
    return softmax_preds[-1] - softmax_preds[0], category

In [7]:
TOXIC_MODELS = {"Cameron/BERT-Jigsaw", 'Cameron/BERT-SBIC-offensive', 
                #'Cameron/BERT-jigsaw-identityhate', 'Cameron/BERT-jigsaw-severetoxic'
               }
GENDER_MODELS = {'Cameron/BERT-mdgender-convai-binary', 'Cameron/BERT-mdgender-convai-ternary', 'Cameron/BERT-mdgender-wizard', 'Cameron/BERT-rtgender-opgender-annotations'}

In [8]:
import glob
FILES = glob.glob("results/Cameron*")

In [9]:
print(FILES[:3])

['results/Cameron-BERT-mdgender-wizard-09:24:57-billsum-train-eval.out', 'results/Cameron-BERT-mdgender-convai-binary-00:42:04-air_dialogue-train-eval.out', 'results/Cameron-BERT-eec-emotion-01:06:29-tweet_eval-train-eval.out']


In [10]:
eval_files = {}
toxic_files = []
gender_files = []

def create_tup(filename, model_name):
    return (model_name.replace("Cameron/", ""), filename)

for filename in FILES:
    s = filename[len("results/"):].split(":")
    
    model_name = s[0][:-3]
    model_name = model_name.replace("Cameron-", "Cameron/")
    eval_dataset_name = s[-1][3:-15]
    model_type = None
    if eval_dataset_name not in eval_files:
        eval_files[eval_dataset_name] = {"toxic": [], "gender": []}
    if model_name in TOXIC_MODELS:
        model_type = "TOXIC"
        eval_files[eval_dataset_name]["toxic"].append(create_tup(filename, model_name))
        toxic_files.append(filename)
    elif model_name in GENDER_MODELS:
        model_type = "GENDER"
        eval_files[eval_dataset_name]["gender"].append(create_tup(filename, model_name))
        gender_files.append(filename)
    else:
        print(f"Model type is not classified, please classify {model_name} with file {filename}")
        continue
    print(f"Model name: {model_name} | eval dataset: {eval_dataset_name}" )

Model name: Cameron/BERT-mdgender-wizard | eval dataset: billsum
Model name: Cameron/BERT-mdgender-convai-binary | eval dataset: air_dialogue
Model type is not classified, please classify Cameron/BERT-eec-emotion with file results/Cameron-BERT-eec-emotion-01:06:29-tweet_eval-train-eval.out
Model name: Cameron/BERT-SBIC-offensive | eval dataset: pubmed_qa
Model type is not classified, please classify Cameron/BERT-jigsaw-severetoxic with file results/Cameron-BERT-jigsaw-severetoxic-00:42:01-air_dialogue-train-eval.out
Model type is not classified, please classify Cameron/BERT-SBIC-targetcategory with file results/Cameron-BERT-SBIC-targetcategory-00:42:18-ted_talks_iwslt-train-eval.out
Model name: Cameron/BERT-Jigsaw | eval dataset: empathetic_dialogues
Model type is not classified, please classify Cameron/BERT-jigsaw-identityhate with file results/Cameron-BERT-jigsaw-identityhate-00:42:20-ted_talks_iwslt-train-eval.out
Model name: Cameron/BERT-mdgender-convai-ternary | eval dataset: conv

In [11]:
print(f"Found {len(toxic_files)} toxic results and {len(gender_files)} gender files")

Found 20 toxic results and 41 gender files


In [12]:
eval_files

{'billsum': {'toxic': [('BERT-Jigsaw',
    'results/Cameron-BERT-Jigsaw-09:27:33-billsum-train-eval.out'),
   ('BERT-SBIC-offensive',
    'results/Cameron-BERT-SBIC-offensive-09:27:16-billsum-train-eval.out')],
  'gender': [('BERT-mdgender-wizard',
    'results/Cameron-BERT-mdgender-wizard-09:24:57-billsum-train-eval.out'),
   ('BERT-rtgender-opgender-annotations',
    'results/Cameron-BERT-rtgender-opgender-annotations-09:24:49-billsum-train-eval.out'),
   ('BERT-mdgender-convai-ternary',
    'results/Cameron-BERT-mdgender-convai-ternary-09:27:16-billsum-train-eval.out'),
   ('BERT-mdgender-convai-binary',
    'results/Cameron-BERT-mdgender-convai-binary-09:27:16-billsum-train-eval.out')]},
 'air_dialogue': {'toxic': [('BERT-SBIC-offensive',
    'results/Cameron-BERT-SBIC-offensive-00:41:58-air_dialogue-train-eval.out'),
   ('BERT-Jigsaw',
    'results/Cameron-BERT-Jigsaw-00:41:54-air_dialogue-train-eval.out')],
  'gender': [('BERT-mdgender-convai-binary',
    'results/Cameron-BERT-md

In [13]:
CACHE = {}
def get_cache(filename):
    if filename in CACHE:
        return CACHE[filename]
    return None

def store_cache(filename, data):
    CACHE[filename] = data
    
def cross_tab_gender_toxic(eval_file_split, eval_dataset_name, return_one = False):
    global CACHE
    CACHE = {}
    eval_crosstabs = []
    count = 0.0
    tot = len(eval_file_split["toxic"]) * len(eval_file_split["gender"])
    for toxic_tup in eval_file_split["toxic"]:
        toxic_model_name = toxic_tup[0]
        toxic_filename = toxic_tup[1]
        for gender_tup in eval_file_split["gender"]:
            gender_model_name = gender_tup[0]
            gender_filename = gender_tup[1]
            
            # for using sbc and rt_gender for conv_ai
            _, _, combined = read_dfs(toxic_filename, 
                                      gender_filename,
                                     suffixes=("_" + toxic_model_name, "_" + gender_model_name))
            pred_toxic = "predictions_" + toxic_model_name
            pred_gender = "predictions_" + gender_model_name
            
            print("relabeling toxic")
            s = get_cache(toxic_filename)
            if s is None:
                print(f"did not find {toxic_filename} in cache")
                p = partial(score, MODEL_RELABEL[toxic_model_name])
                s = combined[pred_toxic].swifter.apply(p)
                scores_col = f"scores_{toxic_model_name}"
                category_col = f"category_{toxic_model_name}"
            
                s = pd.DataFrame(s.tolist(), columns=[scores_col, category_col])
                s[scores_col] = s[scores_col].swifter.apply(lambda x: x.item()) # comes back as a tensor, change it to float
                store_cache(toxic_filename, s)
            else:
                print(f"found toxic {toxic_filename} in cache")
            combined = combined.join(s)
            
            print("relabeling gender")
            s = get_cache(gender_filename)
            if s is None:
                print(f"did not find {gender_filename} in cache")
                p = partial(score, MODEL_RELABEL[gender_model_name])
                s = combined[pred_gender].swifter.apply(p)
                scores_col = f"scores_{gender_model_name}"
                category_col = f"category_{gender_model_name}"
            
                s = pd.DataFrame(s.tolist(), columns=[scores_col, category_col])
                s[scores_col] = s[scores_col].swifter.apply(lambda x: x.item()) # comes back as a tensor, change it to float
                store_cache(gender_filename, s)
            else:
                print(f"found gender {gender_filename} in cache")
            combined = combined.join(s)
            
            
            eval_crosstabs.append({
                "toxic_model_name": toxic_model_name,
                "gender_model_name": gender_model_name,
                "toxic_attr": (f"scores_{toxic_model_name}", f"category_{toxic_model_name}"),
                "gender_attr": (f"scores_{gender_model_name}", f"category_{gender_model_name}"),
                "df": combined
            })
            count += 1.0
            print(f"Crossing {toxic_model_name} x {gender_model_name} for {eval_dataset_name}, finished {count/tot*100}%")
            if return_one:
                return eval_crosstabs
    return eval_crosstabs

In [14]:
eval_files.keys()

dict_keys(['billsum', 'air_dialogue', 'tweet_eval', 'pubmed_qa', 'ted_talks_iwslt', 'empathetic_dialogues', 'conv_ai_3', 'wikipedia', 'yahoo_answers_topics'])

In [15]:
notebook_files = ['tweet_eval', 'pubmed_qa', 'ted_talks_iwslt',]
for eval_dataset_name in notebook_files:
    eval_f = eval_files[eval_dataset_name]
#     if eval_dataset_name != "conv_ai_3":
#         continue
#     if eval_dataset_name == "air_dialogue" or eval_dataset_name == "empathetic_dialogues":
#         print(f'ignoring {eval_dataset_name} for now')
#         continue
    print(f"Analyzing {eval_dataset_name}")
    try:
        ct = cross_tab_gender_toxic(eval_f, eval_dataset_name, return_one=False)
        pickle.dump((eval_dataset_name, ct), open(f"crosstabs-pickle-{eval_dataset_name}.p", "wb"))
        #crosstabs.append((eval_dataset_name, ct))

    except Exception as e:
        print(e)
        raise 
    

Analyzing tweet_eval
relabeling toxic
did not find results/Cameron-BERT-SBIC-offensive-01:08:26-tweet_eval-train-eval.out in cache


Pandas Apply:   0%|          | 0/189034 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/189034 [00:00<?, ?it/s]

relabeling gender
did not find results/Cameron-BERT-mdgender-convai-ternary-00:44:00-tweet_eval-train-eval.out in cache


Pandas Apply:   0%|          | 0/189034 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/23850 [00:00<?, ?it/s]

relabeling gender
found gender results/Cameron-BERT-mdgender-wizard-01:14:09-ted_talks_iwslt-train-eval.out in cache
Crossing BERT-SBIC-offensive x BERT-mdgender-wizard for ted_talks_iwslt, finished 53.125%
relabeling toxic
found toxic results/Cameron-BERT-SBIC-offensive-00:42:20-ted_talks_iwslt-train-eval.out in cache
relabeling gender
found gender results/Cameron-BERT-mdgender-wizard-00:42:15-ted_talks_iwslt-train-eval.out in cache
Crossing BERT-SBIC-offensive x BERT-mdgender-wizard for ted_talks_iwslt, finished 56.25%
relabeling toxic
found toxic results/Cameron-BERT-SBIC-offensive-00:42:20-ted_talks_iwslt-train-eval.out in cache
relabeling gender
found gender results/Cameron-BERT-rtgender-opgender-annotations-01:17:11-ted_talks_iwslt-train-eval.out in cache
Crossing BERT-SBIC-offensive x BERT-rtgender-opgender-annotations for ted_talks_iwslt, finished 59.375%
relabeling toxic
found toxic results/Cameron-BERT-SBIC-offensive-00:42:20-ted_talks_iwslt-train-eval.out in cache
relabeling

Pandas Apply:   0%|          | 0/23850 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23850 [00:00<?, ?it/s]

relabeling gender
found gender results/Cameron-BERT-mdgender-wizard-01:14:09-ted_talks_iwslt-train-eval.out in cache
Crossing BERT-SBIC-offensive x BERT-mdgender-wizard for ted_talks_iwslt, finished 78.125%
relabeling toxic
found toxic results/Cameron-BERT-SBIC-offensive-01:31:48-ted_talks_iwslt-train-eval.out in cache
relabeling gender
found gender results/Cameron-BERT-mdgender-wizard-00:42:15-ted_talks_iwslt-train-eval.out in cache
Crossing BERT-SBIC-offensive x BERT-mdgender-wizard for ted_talks_iwslt, finished 81.25%
relabeling toxic
found toxic results/Cameron-BERT-SBIC-offensive-01:31:48-ted_talks_iwslt-train-eval.out in cache
relabeling gender
found gender results/Cameron-BERT-rtgender-opgender-annotations-01:17:11-ted_talks_iwslt-train-eval.out in cache
Crossing BERT-SBIC-offensive x BERT-rtgender-opgender-annotations for ted_talks_iwslt, finished 84.375%
relabeling toxic
found toxic results/Cameron-BERT-SBIC-offensive-01:31:48-ted_talks_iwslt-train-eval.out in cache
relabeling